In [0]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import re
from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

# #replace with the location of your Part4 folder...
# %cd "/content/drive/My Drive/Colab Notebooks"
# !ls

In [0]:
import keras
from keras.models import Model, Input, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, merge, concatenate
from keras.layers import Activation, Dropout, Flatten, Dense,Input
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU
from keras.callbacks import History

In [0]:
## get all the array values from the npy files 
train_img = np.load('train_images.npy')
train_mask = np.load('train_masks.npy')

In [0]:
########need to resize the images to pass it to U-net 
import random
# indices = np.random.choice(range(len(train_img)), replace = False ,size = 100)

X = train_img[:, :,:,np.newaxis]/255
y= train_mask[:,:,:,np.newaxis]/255

In [0]:
# X = X[:,:,:,np.newaxis] / 255
# y = y[:,:,:,np.newaxis] / 255
print("X shape : ", X.shape)
print("y shape : ", y.shape)

X shape :  (5635, 128, 128, 1)
y shape :  (5635, 128, 128, 1)


In [0]:
def dice_coefficent(img, mask):
    smooth =1
    img_points = K.flatten(img)
    mask_points =K.flatten(mask)
    intersection = K.sum(img_points*mask_points)
    
    coef = (2.*intersection + smooth) / (K.sum(img_points)+K.sum(mask_points) +smooth)
    return coef

def loss_dice(img,mask):
    loss = - dice_coefficent(img,mask)
    return loss
    

In [0]:
IMG_HEIGHT =128
IMG_WIDTH = 128

inputs = Input((IMG_HEIGHT, IMG_WIDTH,1))

conv1 = Conv2D(32, (7, 7), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (5, 5), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (7, 7), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (5, 5), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (7, 7), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (5, 5), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (7, 7), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (5, 5), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (7, 7), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (5, 5), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (7, 7), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (5, 5), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (7, 7), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (5,5), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (3,3), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (7,7), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (5,5), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (3,3), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (7,7), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (5,5), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

# model = Model(inputs=[inputs], outputs=[conv10])
# model.compile(optimizer=Adam(lr = 1e-5), loss=loss_dice, metrics=[dice_coefficent])


# def unet(pretrained_weights = None,input_size = (256,256,3)):
# inputs = Input((IMG_HEIGHT, IMG_WIDTH,1))
# conv1 = Conv2D(64, 5, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
# conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
# # pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
# conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
# conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
# # pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
# conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
# conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
# # pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
# conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
# conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
# # drop4 = Dropout(0.5)(conv4)
# pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

# conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
# conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
# drop5 = Dropout(0.5)(conv5)

# up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
# merge6 = concatenate([drop4,up6], axis = 3)
# conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
# conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

# up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
# merge7 = concatenate([conv3,up7], axis = 3)
# conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
# conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

# up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
# merge8 = concatenate([conv2,up8], axis = 3)
# conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
# conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

# up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
# merge9 = concatenate([conv1,up9], axis = 3)
# conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
# conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
# conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9) # (merge9)
# conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

model = Model(input = inputs, output = conv10)

model.compile(optimizer = Adam(lr = 1e-5), loss = loss_dice, metrics=[dice_coefficent])



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:89: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [0]:
results = model.fit(X, y, validation_split=0.2,batch_size=4, epochs=20,shuffle=True)

Train on 80 samples, validate on 20 samples
Epoch 1/20
80/80 [==============================] - 17s 214ms/step - loss: -0.0240 - dice_coefficent: 0.0240 - val_loss: -0.0236 - val_dice_coefficent: 0.0236
Epoch 2/20
80/80 [==============================] - 17s 218ms/step - loss: -0.0241 - dice_coefficent: 0.0241 - val_loss: -0.0236 - val_dice_coefficent: 0.0236
Epoch 3/20
80/80 [==============================] - 17s 218ms/step - loss: -0.0242 - dice_coefficent: 0.0242 - val_loss: -0.0237 - val_dice_coefficent: 0.0237
Epoch 4/20
80/80 [==============================] - 17s 215ms/step - loss: -0.0243 - dice_coefficent: 0.0243 - val_loss: -0.0239 - val_dice_coefficent: 0.0239
Epoch 5/20
80/80 [==============================] - 17s 210ms/step - loss: -0.0245 - dice_coefficent: 0.0245 - val_loss: -0.0242 - val_dice_coefficent: 0.0242
Epoch 6/20
80/80 [==============================] - 17s 216ms/step - loss: -0.0249 - dice_coefficent: 0.0249 - val_loss: -0.0246 - val_dice_coefficent: 0.0246
Ep

In [0]:
def Generator(X_list, y_list, batch_size = 16):
    while(True):
      X = np.empty((batch_size, 128, 128,1), dtype = 'float32')
      y = np.empty((batch_size, 128, 128,1), dtype = 'float32')
      c = 0

      for i in range(c,c+batch_size):
          X[i - c] = X_list[i]
          y[i - c] = y_list[i]

      c += batch_size
      if(c+batch_size >= len(X_list)):
          c = 0
      yield X, y   

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state = 1)

epochs = 10
batch_size = 16
steps_per_epoch = int(len(X_train) / batch_size)
validation_steps = int(len(X_val) / batch_size)
print(validation_steps)
train_gen = Generator(X_train, y_train, batch_size = batch_size)
val_gen = Generator(X_val, y_val, batch_size = batch_size)

model = Model(inputs=[inputs], outputs=[conv10])
model.compile(optimizer=Adam(lr = 1e-4), loss=loss_dice, metrics=[dice_coefficent])


70


In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4, min_lr=1e-6)


history = model.fit_generator(train_gen, steps_per_epoch=steps_per_epoch, epochs = epochs,
                             validation_data = val_gen, validation_steps = validation_steps,callbacks= [reduce_lr])

Epoch 1/10
281/281 [==============================] - 90s 319ms/step - loss: -0.1982 - dice_coefficent: 0.1982 - val_loss: -0.4891 - val_dice_coefficent: 0.4891
Epoch 2/10
281/281 [==============================] - 80s 286ms/step - loss: -0.7085 - dice_coefficent: 0.7085 - val_loss: -0.4956 - val_dice_coefficent: 0.4956
Epoch 3/10
281/281 [==============================] - 80s 285ms/step - loss: -0.9322 - dice_coefficent: 0.9322 - val_loss: -0.5675 - val_dice_coefficent: 0.5675
Epoch 4/10
281/281 [==============================] - 80s 285ms/step - loss: -0.9732 - dice_coefficent: 0.9732 - val_loss: -0.5626 - val_dice_coefficent: 0.5626
Epoch 5/10
281/281 [==============================] - 80s 284ms/step - loss: -0.9832 - dice_coefficent: 0.9832 - val_loss: -0.5700 - val_dice_coefficent: 0.5700
Epoch 6/10
281/281 [==============================] - 80s 284ms/step - loss: -0.9892 - dice_coefficent: 0.9892 - val_loss: -0.5875 - val_dice_coefficent: 0.5875
Epoch 7/10
281/281 [==============

In [0]:
# test_list = os.listdir("input/test/test")
# reg =re.compile("[0-9]+")

# temp1 = list(map(lambda x: reg.match(x).group(), test_list)) 
# temp1 = list(map(int, temp1))
# test_list = [x for _,x in sorted(zip(temp1, test_list))]
# X_test=[]
# for i in range(len(test_list)):
#     img = Image.open('input/test/test/'+test_list[i])
#     img_resized = img.resize((128,128))
#     X_test.append(np.array(img_resized))
    
# X_test= np.asarray(X_test)
# X= X[:,:,:,np.newaxis]/255

X_test = np.load('xtest.npy')

In [0]:
y_pred =model.predict(X_test)

In [0]:
def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])


rles = []
for i in range(X_test.shape[0]):
    img = y_pred[i, :, :, 0]
    img = img > 0.5
    img = resize(img, (420, 580), preserve_range=True)
    rle = run_length_enc(img)
    rles.append(rle)
    if i % 100 == 0:
        print('{}/{}'.format(i, X_test.shape[0]), end = "\r")
         
sub = pd.read_csv('sample_submission.csv')
sub['pixels'] = rles
sub.to_csv("submission4.csv", index = False)